In [8]:
# Our function library
import functions as f

In [9]:
# Making test data
make_test_data('magnus_bullet_50_games.json', 'DrNykterstein', 5, 'bullet')
make_test_data('Dan_blitz_games.json', 'Captain_Brunch', 5, 'blitz')

In [10]:
# Import test data
import json
dict_list = json.load(open('Dan_blitz_games.json'))
game_dict = dict_list[0]